# Time dependent D3D simulation

In [ ]:
using Plots;
using FUSE
FUSE.ProgressMeter.ijulia_behavior(:clear);

In [ ]:
ini, act = FUSE.case_parameters(:D3D, :default)
FUSE.init(ini, act);

In [ ]:
#ini, act = FUSE.case_parameters(:D3D, 180893); #NBI with balanced torque
#ini, act = FUSE.case_parameters(:D3D, 133221); # ECH
#ini, act = FUSE.case_parameters(:D3D, 200000); # ECH
#ini, act = FUSE.case_parameters(:D3D, 170325); # ECH 1.67924
ini, act = FUSE.case_parameters(:D3D, 168830);
#ini, act = FUSE.case_parameters(:D3D, 200204); # beam power scan, rotation, lower_to_upper null
#ini, act = FUSE.case_parameters(:D3D, 200021; EFIT_tree="EFIT01"); # negative B
@checkin :fetch ini act

In [ ]:
@checkout :fetch ini act
using Interact
@manipulate for time0 in slider(ini.general.dd.equilibrium.time, value=ini.general.dd.global_time, label="time")
    # p=plot(ini.general.dd.equilibrium; time0)
    # plot!(p[1], ini.general.dd.wall)

    plot(ini.general.dd.core_profiles; time0)
end

In [ ]:
#plot(ini.general.dd.nbi; smooth_tau=0.1)
#plot(ini.general.dd.ec_launchers; smooth_tau=0.1)

In [ ]:
@checkout :fetch ini act
ini.time.simulation_start = ini.time.pulse_shedule_time_basis[2]
@show ini.time.simulation_start
dd = IMAS.dd()
FUSE.init(dd, ini, act); # populates dd.pulse_schedule based on ini.general.dd
act.ActorReplay.replay_dd = deepcopy(dd)
@checkin :init dd ini act;

In [ ]:
@checkout :init dd ini act;
using Interact
@manipulate for time0 in slider(dd.equilibrium.time, value=dd.global_time, label="time")
    p = plot(dd.equilibrium; time0)
    plot!(p[1], dd.wall)

    plot(dd.core_profiles; time0)
end

In [ ]:
@checkout :init dd ini act;
LH_dyanamics = FUSE.ne_line_without_LH_transition(dd, 1.8, 2.2, 2.2; do_plot=true);
#LH_dyanamics = FUSE.ne_line_without_LH_transition(dd, 1.175, 1.5, 1.4; do_plot=true);

If the simulation is starting from experimental conditions (ie. we have not changed the equilibrium, profiles, sources when the time dependent simulation starts), our plasma should already be in a self-consistent state and we are ready to start our time dependent simulation. If the simulation does not start from know experimental conditions, we would need to call the `ActorStationaryPlasma` to start from a know consistent state.

In [ ]:
plot(ini.general.dd.core_profiles.time, [cp1d.zeff[1] for cp1d in ini.general.dd.core_profiles.profiles_1d])

In [ ]:
LH_dyanamics.mode_transitions

In [ ]:
@checkout :init dd act;
act.ActorPedestal.model = :dynamic
act.ActorPedestal.tau_n = LH_dyanamics.tau_n
act.ActorPedestal.tau_t = LH_dyanamics.tau_t
act.ActorWPED.ped_to_core_fraction = LH_dyanamics.W_ped_to_core_fraction
act.ActorPedestal.mode_transitions = LH_dyanamics.mode_transitions
if false
    # density from experiment
    act.ActorPedestal.density_ratio_L_over_H = 1.0
else
    # density can go from L to H mode at a different time
    act.ActorPedestal.density_ratio_L_over_H = LH_dyanamics.ne_L_over_H
    dd.pulse_schedule.density_control.n_e_line.reference = LH_dyanamics.ne_H
end

act.ActorEPED.ped_factor = 0.8
act.ActorPedestal.T_ratio_pedestal = 1.0 # Ti/Te in the pedestal

δt = 0.025
final_time = 2.0
act.ActorDynamicPlasma.Nt = Int(ceil((final_time - dd.global_time) / δt))# * 0
act.ActorDynamicPlasma.Δt = final_time - dd.global_time

act.ActorDynamicPlasma.evolve_current = true
act.ActorDynamicPlasma.evolve_equilibrium = true
act.ActorDynamicPlasma.evolve_transport = true
act.ActorDynamicPlasma.evolve_hcd = true
act.ActorDynamicPlasma.evolve_pf_active = false
act.ActorDynamicPlasma.evolve_pedestal = true

# act.ActorDynamicPlasma.ip_controller = false
act.ActorDynamicPlasma.time_derivatives_sources = true

act.ActorTGLF.tglfnn_model = "sat2_em_d3d"
act.ActorFluxMatcher.verbose = true

act.ActorCurrent.model = :replay
act.ActorEquilibrium.model = :replay
act.ActorCoreTransport.model = :replay
act.ActorPedestal.model = :replay
# act.ActorHCD.ec_model = :replay
# act.ActorHCD.ic_model = :replay
# act.ActorHCD.lh_model = :replay
# act.ActorHCD.nb_model = :replay
# act.ActorHCD.pellet_model = :replay

actor = FUSE.ActorDynamicPlasma(dd, act; verbose=true);

@checkin :time_dep_dt1 dd ini act actor;

In [ ]:
@checkout :time_dep_dt1 dd act
using Interact
@manipulate for time0 in slider(dd.equilibrium.time, value=dd.global_time, label="time")
    # #FUSE.plot_plasma_overview(dd, time0; dd1=FUSE.checkpoint[:init].dd, size=(1400,1000))

    plot(dd.core_sources; time0)

    # p=plot(FUSE.checkpoint[:init].dd.core_profiles; time0, color=:black)
    # plot!(dd.core_profiles; time0)

    # plot(FUSE.checkpoint[:init].dd.core_profiles.profiles_1d[time0], :zeff; time0, color=:black)
    # plot!(dd.core_profiles.profiles_1d[time0], :zeff)
    # vline!(p[2], [0.9])
    # #plot!(p[2], ylim=(0, 8E19))

    # p=plot(FUSE.checkpoint[:init].dd.equilibrium; time0, color=:black)
    # plot!(dd.equilibrium; time0)
end

Now we can plot the results, either as a animated gif

In [ ]:
@checkout :time_dep_dt1 dd ini act actor;

times = dd.core_profiles.time[2:end]
prog = FUSE.ProgressMeter.Progress(length(times))
for (k, time0) in enumerate(times)
    FUSE.ProgressMeter.next!(prog; showvalues=(("k", k), ("time", time0)))
    if k < 134
        continue
    end
    FUSE.plot_plasma_overview(FUSE.checkpoint[:time_dep_dt1].dd, time0; dd1=FUSE.checkpoint[:init].dd)
    IMAS.ylim(Dict{Int,Float64}(
        3 => 3.0, 4 => 1E20,
        -6 => -0.5, 6 => 1.5, -7 => -0.5, 7 => 1.5, -8 => -2E20, 8 => 2.5E20,
        -10 => 0.0, 10 => 0.101, -11 => 0.0, 11 => 0.101, -12 => -1.2E19, 12 => 1.2E19))
    savefig("D3D_168830_rampdown/D3D_time_dep__$k.png")
end

In [ ]:
time0 = 1.0
FUSE.plot_plasma_overview(FUSE.checkpoint[:time_dep_dt1].dd, time0; dd1=FUSE.checkpoint[:init].dd)